In [8]:
# dependencies
import io
from datetime import datetime, date, timedelta
import xarray as xr
import requests
import matplotlib.pyplot as plt
from herbie import Herbie

# Not used directly, but used via xarray
import cfgrib

 ╭─▌▌Herbie─────────────────────────────────────────────╮
 │ INFO: Created a default config file.                 │
 │ You may view/edit Herbie's configuration here:       │
 │     C:\Users\b25ch\.config\herbie\config.toml        │
 ╰──────────────────────────────────────────────────────╯



In [29]:
# Constants for creating the full URL
base_url = "https://noaahrrr.blob.core.windows.net/hrrr"
sector = "conus"
yesterday = date.today() - timedelta(days=1)
cycle = 12          # noon
forecast_hour = 0   # offset from cycle time
product = "wrfsfcf" # 2D surface levels

# Put it all together
file_path = f"hrrr.t{cycle:02}z.{product}{forecast_hour:02}.grib2"
url = f"{base_url}/hrrr.{yesterday:%Y%m%d}/{sector}/{file_path}"

print(url)

https://noaahrrr.blob.core.windows.net/hrrr/hrrr.20240708/conus/hrrr.t12z.wrfsfcf00.grib2


In [33]:
# Fetch the idx file by appending the .idx file extension to our already formatted URL
r = requests.get(f"{url}.idx")
idx = r.text.splitlines()
# Take a peek at the content of the index
print(*idx[0:10], sep="\n")

1:0:d=2024070812:REFC:entire atmosphere:anl:
2:262634:d=2024070812:RETOP:cloud top:anl:
3:450410:d=2024070812:var discipline=0 center=7 local_table=1 parmcat=16 parm=201:entire atmosphere:anl:
4:717802:d=2024070812:VIL:entire atmosphere:anl:
5:935013:d=2024070812:VIS:surface:anl:
6:2266565:d=2024070812:REFD:1000 m above ground:anl:
7:2401864:d=2024070812:REFD:4000 m above ground:anl:
8:2549182:d=2024070812:REFD:263 K level:anl:
9:2702127:d=2024070812:GUST:surface:anl:
10:3932638:d=2024070812:UGRD:250 mb:anl:


In [34]:
# You can see it has a 1-indexed base line number, staring byte position, date, variable, atmosphere level,
# and forecast description. The lines are colon-delimited. 

# Let's grab surface temperature `TMP:surface`.
sfc_precip_idx = [l for l in idx if ":APCP:" in l][0].split(":")
print("Precipitation:", sfc_precip_idx)

# Pluck the byte offset from this line, plus the beginning offset of the next line
line_num = int(sfc_precip_idx[0])
range_start = sfc_precip_idx[1]

# The line number values are 1-indexed, so we don't need to increment it to get the next list index,
# but check we're not already reading the last line
next_line = idx[line_num].split(':') if line_num < len(idx) else None

# Pluck the start of the next byte offset, or nothing if we were on the last line
range_end = next_line[1] if next_line else None

print(f"Byte range: {range_start}-{range_end}")

Precipitation: ['84', '50752277', 'd=2024070812', 'APCP', 'surface', '0-0 day acc fcst', '']
Byte range: 50752277-50752489


In [9]:
# Herbie object for the HRRR model 6-hr surface forecast product
H = Herbie(
  '2021-01-01',
  product='sfc'
)

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2021-Jan-01 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


In [20]:
H.inventory(search=":APCP:")

,grib_message,start_byte,end_byte,range,reference_time,valid_time,variable,level,forecast_time,search_this
83,84,53837882,53838093.0,53837882-53838093,2021-01-01,2021-01-01,APCP,surface,0-0 day acc fcst,:APCP:surface:0-0 day acc fcst


In [23]:
ds = H.xarray(":APCP:")
ds

<xarray.Dataset>
Dimensions:              (y: 1059, x: 1799)
Coordinates:
    time                 datetime64[ns] 2021-01-01
    step                 timedelta64[ns] 00:00:00
    surface              float64 0.0
    latitude             (y, x) float64 21.14 21.15 21.15 ... 47.86 47.85 47.84
    longitude            (y, x) float64 237.3 237.3 237.3 ... 299.0 299.0 299.1
    valid_time           datetime64[ns] 2021-01-01
    gribfile_projection  object None
Dimensions without coordinates: y, x
Data variables:
    tp                   (y, x) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP 
    model:                   hrrr
    product:                 sfc
    description:             High-Resolution Rapid Refresh - CONUS
    remote_grib:             https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr....
    local_grib:              C:\Users\b25ch\data\hrrr\20210101\subset_abef2f9...
    search:                  :APCP:

In [19]:
rain = ds.tp
rain

<xarray.DataArray 'tp' (y: 1059, x: 1799)>
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)
Coordinates:
    time                 datetime64[ns] 2021-01-01
    step                 timedelta64[ns] 00:00:00
    surface              float64 0.0
    latitude             (y, x) float64 21.14 21.15 21.15 ... 47.86 47.85 47.84
    longitude            (y, x) float64 237.3 237.3 237.3 ... 299.0 299.0 299.1
    valid_time           datetime64[ns] 2021-01-01
    gribfile_projection  object None
Dimensions without coordinates: y, x
Attributes: (12/38)
    GRIB_paramId:                             228228
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      1905141
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            accum
    ...                                       ...
    GRIB_units:                               kg m**-2
    GRIB_uvRelativeToGrid:                    1
    long_name:                                Total Precipitation
    units:                                    kg m**-2
    standard_name:                            unknown
    grid_mapping:                             gribfile_projection